In [1]:
# Cell 1: Mount Google Drive & set paths

from google.colab import drive
drive.mount('/content/drive')

from pathlib import Path
import os
import pandas as pd

project_dir = Path("/content/drive/MyDrive/CVproject/stage1")
project_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path("/content/drive/MyDrive/ColabData/datasets/FEC_project")
data_dir.mkdir(parents=True, exist_ok=True)

meta_dir = data_dir / "meta"
images_drive_dir = data_dir / "images"

print("project_dir:", project_dir)
print("data_dir   :", data_dir)
print("meta_dir   :", meta_dir)
print("images_dir :", images_drive_dir)

print("meta files:", os.listdir(meta_dir))
print("images subdirs:", os.listdir(images_drive_dir))


Mounted at /content/drive
project_dir: /content/drive/MyDrive/CVproject/stage1
data_dir   : /content/drive/MyDrive/ColabData/datasets/FEC_project
meta_dir   : /content/drive/MyDrive/ColabData/datasets/FEC_project/meta
images_dir : /content/drive/MyDrive/ColabData/datasets/FEC_project/images
meta files: ['fec_train_clean.csv', 'fec_test_clean.csv', 'fec_train_clean_with_paths.csv', 'fec_train_download_errors.csv', 'fec_test_clean_with_paths.csv', 'fec_test_download_errors.csv', 'fec_train_final.csv', 'fec_test_final.csv', 'fec_train_raw.csv', 'fec_test_raw.csv', 'fec_train_raw_with_paths_clean.csv', 'fec_train_download_errors_detailed.csv', 'fec_test_download_errors_detailed.csv', 'fec_test_raw_with_paths_clean.csv', 'fec_test_raw_with_paths_clean_with_L.csv', 'fec_train_raw_with_paths_clean_with_L.csv']
images subdirs: ['2b8dc78b35805985.jpg', 'e6230eaea9605bcf.jpg', 'f6dadc4315f3a2a7.jpg', 'd6fa55c563d7c2df.jpg', '9526b698a2ec4022.jpg', '9feebba8b3d38ed0.jpg', 'a5dc6cf092ec1de8.jpg', 

In [3]:

from pathlib import Path
import shutil
import pandas as pd
import time

t0 = time.time()
print("Planning to use a local CSV instead of reading the Excel file on Drive")

print("data_dir  :", data_dir)
print("meta_dir  :", meta_dir)

#Original xlsx spreadsheet on Drive (keep the directory unchanged)
train_xlsx = data_dir / "faceexp-comparison-data-train-public.xlsx"
test_xlsx  = data_dir / "faceexp-comparison-data-test-public.xlsx"

if not train_xlsx.exists() or not test_xlsx.exists():
    raise FileNotFoundError(
        f"The FEC xlsx file cannot be found. Please confirm that it has been downloaded to: {data_dir}"
    )

#local root
local_root       = Path("/content/fec_local")
local_meta_dir   = local_root / "meta"
local_images_dir = local_root / "images"   #It is also recommended to place the cropped images downloaded later her

for p in [local_root, local_meta_dir, local_images_dir]:
    p.mkdir(parents=True, exist_ok=True)

print("Local working directory:", local_root)

train_csv_drive = meta_dir / "fec_train_raw.csv"
test_csv_drive  = meta_dir / "fec_test_raw.csv"

if (not train_csv_drive.exists()) or (not test_csv_drive.exists()):
    print("First run: Read from Excel and save as csv (placed in Drive/meta)")
    df_train_raw = pd.read_excel(train_xlsx)
    df_test_raw  = pd.read_excel(test_xlsx)

    df_train_raw.to_csv(train_csv_drive, index=False)
    df_test_raw.to_csv(test_csv_drive, index=False)

    print("The original csv has been saved in Drive：")
    print("   ", train_csv_drive)
    print("   ", test_csv_drive)
else:
    print("fec_train_raw.csv / fec_test_raw.csv are already on Drive; skipping Excel reading.")

#Copy the csv file in Drive/meta to the local machine, and all subsequent operations will be based on the local csv file.
local_train_csv = local_meta_dir / train_csv_drive.name
local_test_csv  = local_meta_dir / test_csv_drive.name

shutil.copy2(train_csv_drive, local_train_csv)
shutil.copy2(test_csv_drive, local_test_csv)

print("The csv has been copied to the local machine.:", local_meta_dir)

df_train_raw = pd.read_csv(local_train_csv)
df_test_raw  = pd.read_csv(local_test_csv)

print("train triples:", len(df_train_raw))
print("test  triples:", len(df_test_raw))
print(f"Total time consumed by this cell: {time.time() - t0:.1f} s")

raw_train_meta_local = local_train_csv
raw_test_meta_local  = local_test_csv


Planning to use a local CSV instead of reading the Excel file on Drive
data_dir  : /content/drive/MyDrive/ColabData/datasets/FEC_project
meta_dir  : /content/drive/MyDrive/ColabData/datasets/FEC_project/meta
Local working directory: /content/fec_local
fec_train_raw.csv / fec_test_raw.csv are already on Drive; skipping Excel reading.
The csv has been copied to the local machine.: /content/fec_local/meta
train triples: 449143
test  triples: 51060
Total time consumed by this cell: 3.5 s


In [4]:
#Download locally in Colab + crop + download while cleaning invalid triplets

import os
from pathlib import Path
from io import BytesIO
from concurrent.futures import ThreadPoolExecutor, as_completed

import numpy as np
import pandas as pd
import requests
from PIL import Image
from tqdm.notebook import tqdm

print("local_root:", local_root)
print("meta_dir  :", meta_dir)

images_dir_local    = local_root / "images"
train_img_dir_local = images_dir_local / "train"
test_img_dir_local  = images_dir_local / "test"
os.makedirs(train_img_dir_local, exist_ok=True)
os.makedirs(test_img_dir_local,  exist_ok=True)

print("Local images root:", images_dir_local)
print("  train:", train_img_dir_local)
print("  test :", test_img_dir_local)


def download_image(url: str, timeout: int = 10) -> Image.Image:
    resp = requests.get(url, timeout=timeout)
    resp.raise_for_status()
    img = Image.open(BytesIO(resp.content)).convert("RGB")
    return img


def crop_with_norm_bbox_xyxy(
    img: Image.Image,
    left_x: float, right_x: float,
    top_y: float, bottom_y: float,
    out_size: int = 224,
) -> Image.Image:
    """
    Crop according to the normalized bbox (left, right, top, bottom), then resize to out_size x out_size
    The order in the FEC document is: left_col, right_col, top_row, bottom_row (all are proportional coordinates in [0,1])
    """
    w, h = img.size

    x1 = int(round(left_x * w))
    x2 = int(round(right_x * w))
    y1 = int(round(top_y * h))
    y2 = int(round(bottom_y * h))

    # Boundary clipping
    x1 = max(0, min(x1, w - 1))
    x2 = max(0, min(x2, w))
    y1 = max(0, min(y1, h - 1))
    y2 = max(0, min(y2, h))

    if x2 <= x1 or y2 <= y1:
        raise ValueError(f"Invalid bbox after scaling: {(x1, y1, x2, y2)}, img size={(w, h)}")

    face = img.crop((x1, y1, x2, y2))
    face = face.resize((out_size, out_size), Image.BILINEAR)
    return face


def get_image_url_and_bbox_cols(df: pd.DataFrame, k: int):
    """
    For the k-th image (k=1,2,3), return (url_col_name, bbox_col_names_list)

    The column order of FEC (whether in csv or xlsx format) is follows:
      [url1, left1, right1, top1, bottom1,
       url2, left2, right2, top2, bottom2,
       url3, left3, right3, top3, bottom3, ...]
    So here, we retrieve based on the column position, regardless of the specific column name.
    """
    base = (k - 1) * 5
    cols = list(df.columns)[base: base + 5]
    url_col = cols[0]
    bbox_cols = cols[1:]
    return url_col, bbox_cols


def save_crop_one_image(url: str, bbox_vals, out_path: Path, timeout: int = 10):
    #Download, crop, and save a face image.
    img = download_image(url, timeout=timeout)
    left_x, right_x, top_y, bottom_y = map(float, bbox_vals)
    face = crop_with_norm_bbox_xyxy(img, left_x, right_x, top_y, bottom_y)
    out_path.parent.mkdir(parents=True, exist_ok=True)
    face.save(str(out_path), format="JPEG", quality=95)
    return None


def process_split_named_fast(
    meta_path: Path,
    out_dir: Path,
    split_name: str = "train",
    max_rows: int = None,
    num_workers: int = 16,
):
    """
    Process a split (train / test):
      Read the meta CSV (pass a local CSV here, such as raw_train_meta_local)
      Generate file names for each triplet of 3 images: {split}_{triplet_id:06d}_img{j}.jpg
      If the image does not exist: download + crop + save (multi-threaded)
      If any image in a triplet fails:
      Delete all successfully downloaded images of that triplet
      Remove the corresponding row from the final meta
      Finally, save a new CSV "retaining only valid triplets" to the Drive's meta_dir
      At the same time, save the detailed error information to an error CSV
    """
    print(f"\n=== Processing split: {split_name} ===")
    print("Reading meta from:", meta_path)

    df = pd.read_csv(meta_path)

    if max_rows is not None:
        df = df.iloc[:max_rows].reset_index(drop=True)

    # Assign a clear triplet_id to each line (starting from 1)
    df["triplet_id"] = np.arange(1, len(df) + 1)

    # (url_col, bbox_cols) for each of the three images
    img_specs = []
    for k in [1, 2, 3]:
        url_col, bbox_cols = get_image_url_and_bbox_cols(df, k)
        img_specs.append((url_col, bbox_cols))

    # Record all file paths corresponding to each triplet for deletion after failure
    triplet_files = {int(tid): [] for tid in df["triplet_id"].tolist()}

    tasks = []
    future_to_task = {}

    # Construct a task list
    for idx, row in df.iterrows():
        triplet_id = int(row["triplet_id"])
        for j, (url_col, bbox_cols) in enumerate(img_specs, start=1):
            url = row[url_col]
            bbox_vals = row[list(bbox_cols)].values

            fname = f"{split_name}_{triplet_id:06d}_img{j}.jpg"
            out_path = out_dir / fname

            # Record paths in the DataFrame
            colname = f"img{j}_path"
            df.loc[idx, colname] = str(out_path)

            # Record to triplet_files (even if the file already exists, it will be needed when deleting later)
            triplet_files[triplet_id].append(out_path)

            # Create a download task only when the file does not exist to implement resumable downloads.
            if not out_path.exists():
                tasks.append((triplet_id, j, url, bbox_vals, out_path))

    print(f"{split_name}: triplets in this run = {len(df)}")
    print(f"{split_name}: images to (down)load = {len(tasks)}")

    error_records = []
    failed_triplets = set()

    if tasks:
        with ThreadPoolExecutor(max_workers=num_workers) as ex:
            for (triplet_id, j, url, bbox_vals, out_path) in tasks:
                fut = ex.submit(save_crop_one_image, url, bbox_vals, out_path)
                future_to_task[fut] = (triplet_id, j, url, out_path)

            pbar = tqdm(total=len(future_to_task), desc=f"{split_name} download+crop")

            for fut in as_completed(future_to_task):
                triplet_id, j, url, out_path = future_to_task[fut]
                try:
                    fut.result()
                except Exception as e:
                    # Record the error information and mark the triplet as failed.
                    failed_triplets.add(triplet_id)
                    error_records.append({
                        "triplet_id": triplet_id,
                        "img_index": j,
                        "url": url,
                        "out_path": str(out_path),
                        "error": str(e),
                    })
                    # Prevent residual bad files that have been partially written
                    try:
                        if out_path.exists():
                            out_path.unlink()
                    except Exception:
                        pass
                finally:
                    pbar.update(1)
            pbar.close()

    # For failed triplets: delete all the images that have been successfully downloaded
    if failed_triplets:
        print(f"{split_name}: removing images for {len(failed_triplets)} failed triplets...")
        for t_id in failed_triplets:
            for p in triplet_files.get(t_id, []):
                try:
                    if p.exists():
                        p.unlink()
                except Exception:
                    pass

    # Construct a DataFrame that "only contains valid triplets"
    n_before = len(df)
    if failed_triplets:
        df_clean = df[~df["triplet_id"].isin(failed_triplets)].reset_index(drop=True)
    else:
        df_clean = df.copy()

    n_after = len(df_clean)
    print(f"{split_name}: valid triplets kept = {n_after} / {n_before}")

    # The finally saved meta: remove the triplet_id, only retain the original columns + img*_path columns
    df_clean = df_clean.drop(columns=["triplet_id"])

    out_meta_with_paths = meta_dir / f"{meta_path.stem}_with_paths_clean.csv"
    df_clean.to_csv(out_meta_with_paths, index=False)
    print(f"Saved CLEAN meta with image paths to: {out_meta_with_paths}")

    #Record errors
    if error_records:
        err_path = meta_dir / f"fec_{split_name}_download_errors_detailed.csv"
        pd.DataFrame(error_records).to_csv(err_path, index=False)
        print(f"{split_name}: {len(error_records)} images failed, logged in {err_path}")
    else:
        print(f"{split_name}: no download errors")


#Actually start downloading

process_split_named_fast(
    meta_path=raw_train_meta_local,
    out_dir=train_img_dir_local,
    split_name="train",
    max_rows=80000,
    num_workers=16
)

process_split_named_fast(
    meta_path=raw_test_meta_local,
    out_dir=test_img_dir_local,
    split_name="test",
    max_rows=10000,
    num_workers=16
)


local_root: /content/fec_local
meta_dir  : /content/drive/MyDrive/ColabData/datasets/FEC_project/meta
Local images root: /content/fec_local/images
  train: /content/fec_local/images/train
  test : /content/fec_local/images/test

=== Processing split: train ===
Reading meta from: /content/fec_local/meta/fec_train_raw.csv
train: triplets in this run = 80000
train: images to (down)load = 143166


train download+crop:   0%|          | 0/143166 [00:00<?, ?it/s]

train: removing images for 26034 failed triplets...
train: valid triplets kept = 53966 / 80000
Saved CLEAN meta with image paths to: /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_train_raw_with_paths_clean.csv
train: 40778 images failed, logged in /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_train_download_errors_detailed.csv

=== Processing split: test ===
Reading meta from: /content/fec_local/meta/fec_test_raw.csv
test: triplets in this run = 10000
test: images to (down)load = 26409


test download+crop:   0%|          | 0/26409 [00:00<?, ?it/s]

test: removing images for 3029 failed triplets...
test: valid triplets kept = 6971 / 10000
Saved CLEAN meta with image paths to: /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_test_raw_with_paths_clean.csv
test: 3425 images failed, logged in /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_test_download_errors_detailed.csv


In [5]:
#Use the cleaned meta file with path

clean_train_meta_path = meta_dir / "fec_train_raw_with_paths_clean.csv"
clean_test_meta_path  = meta_dir / "fec_test_raw_with_paths_clean.csv"

print("Clean train meta:", clean_train_meta_path)
print("Clean test  meta:", clean_test_meta_path)


Clean train meta: /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_train_raw_with_paths_clean.csv
Clean test  meta: /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_test_raw_with_paths_clean.csv


In [6]:

import numpy as np
import pandas as pd
from PIL import Image
import torch
from torch.utils.data import Dataset

img_size = 224

def add_label_L_if_missing(df: pd.DataFrame, csv_path=None) -> pd.DataFrame:
    #If there is no 'L' column in df, calculate a majority vote for L ∈ {1,2,3} from multiple Annotation columns.
    if "L" in df.columns:
        return df

    # Find all Annotation columns (the original FEC are Annotation1, Annotation2,)
    ann_cols = [c for c in df.columns if c.lower().startswith("annotation")]
    if not ann_cols:
        raise ValueError(
            f"in {csv_path} can not fine any Annotation column，The current column name is：\n{list(df.columns)}"
        )

    ann_df = df[ann_cols].apply(pd.to_numeric, errors="coerce")  # Convert to a number; if invalid, convert to NaN

    # When calculating the mode row by row, pandas' mode will return the smaller one in case of a tie.
    mode_vals = ann_df.mode(axis=1, numeric_only=True)

    # mode(axis=1) It may return multiple columns, and we will take the first one.
    L = mode_vals.iloc[:, 0].round().astype("Int64")  # allow NaN

    df = df.copy()
    df["L"] = L

    # Filter out those that haven't been calculated (the ones that are all NaN) or those not in {1, 2, 3}
    df = df[df["L"].isin([1, 2, 3])].reset_index(drop=True)
    df["L"] = df["L"].astype(int)

    if csv_path is not None:
        out_with_L = Path(csv_path).with_name(Path(csv_path).stem + "_with_L.csv")
        df.to_csv(out_with_L, index=False)
        print(f"for {csv_path} compute L，saved to: {out_with_L}")

    return df


class FECTripletDataset(Dataset):
    """
    For training: return (anchor, positive, negative)
    Meaning of L (original definition in FEC):
    L = 1 -> (2,3) are relatively similar, and 1 is the "odd one out"
    L = 2 -> (1,3) are similar
    L = 3 -> (1,2) are similar
    We take the "two similar ones" as (anchor, positive), and the other one as negative。
    """
    def __init__(self, csv_path, transform=None, max_retry=5):
        self.csv_path = csv_path
        df = pd.read_csv(csv_path)
        df = add_label_L_if_missing(df, csv_path=csv_path)
        self.df = df
        self.transform = transform
        self.max_retry = max_retry

    def __len__(self):
        return len(self.df)

    def _load_img(self, path):
        img = Image.open(path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img

    def __getitem__(self, idx):
        #Retry randomly several times when there is a bad path
        for _ in range(self.max_retry):
            row = self.df.iloc[idx]
            paths = [row["img1_path"], row["img2_path"], row["img3_path"]]
            L = int(row["L"])

            try:
                imgs = [self._load_img(p) for p in paths]
                #Press L to map a/p/n
                if L == 1:
                    # (2,3) are similar, 1 is odd
                    anchor, positive, negative = imgs[1], imgs[2], imgs[0]
                elif L == 2:
                    # (1,3) are similar, 2 is odd
                    anchor, positive, negative = imgs[0], imgs[2], imgs[1]
                else:  # L == 3
                    # (1,2) are similar, 3 is odd
                    anchor, positive, negative = imgs[0], imgs[1], imgs[2]

                return anchor, positive, negative
            except (FileNotFoundError, OSError):
                # Randomly change a line
                idx = np.random.randint(0, len(self.df))

        # If it really doesn't work, return 0 images to avoid crashing.
        dummy = torch.zeros(3, img_size, img_size)
        return dummy, dummy, dummy


class FECTriEvalDataset(Dataset):
    """
    For evaluation: directly return (img1, img2, img3, L)
    Used to calculate triplet accuracy.
    """
    def __init__(self, csv_path, transform=None, max_retry=5):
        self.csv_path = csv_path
        df = pd.read_csv(csv_path)
        df = add_label_L_if_missing(df, csv_path=csv_path)
        self.df = df
        self.transform = transform
        self.max_retry = max_retry

    def __len__(self):
        return len(self.df)

    def _load_img(self, path):
        img = Image.open(path).convert("RGB")
        if self.transform is not None:
            img = self.transform(img)
        return img

    def __getitem__(self, idx):
        for _ in range(self.max_retry):
            row = self.df.iloc[idx]
            paths = [row["img1_path"], row["img2_path"], row["img3_path"]]
            L = int(row["L"])
            try:
                imgs = [self._load_img(p) for p in paths]
                return imgs[0], imgs[1], imgs[2], torch.tensor(L, dtype=torch.long)
            except (FileNotFoundError, OSError):
                idx = np.random.randint(0, len(self.df))

        dummy = torch.zeros(3, img_size, img_size)
        return dummy, dummy, dummy, torch.tensor(1, dtype=torch.long)


In [7]:
# Construct DataLoader

from torch.utils.data import DataLoader
from torchvision import transforms

imagenet_mean = [0.485, 0.456, 0.406]
imagenet_std  = [0.229, 0.224, 0.225]
img_size = 224

train_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

test_transform = transforms.Compose([
    transforms.Resize((img_size, img_size)),
    transforms.ToTensor(),
    transforms.Normalize(mean=imagenet_mean, std=imagenet_std),
])

batch_size = 128

train_dataset      = FECTripletDataset(clean_train_meta_path, transform=train_transform)
train_eval_dataset = FECTriEvalDataset(clean_train_meta_path, transform=test_transform)
test_eval_dataset  = FECTriEvalDataset(clean_test_meta_path,  transform=test_transform)

print("Train dataset len      :", len(train_dataset))
print("Train eval dataset len :", len(train_eval_dataset))
print("Test  eval dataset len :", len(test_eval_dataset))

num_workers = 8

train_loader = DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=num_workers,
    pin_memory=True,
)

train_eval_loader = DataLoader(
    train_eval_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

test_eval_loader = DataLoader(
    test_eval_dataset,
    batch_size=batch_size,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True,
)

# sanity check
a, p, n = next(iter(train_loader))
print("Anchor batch shape  :", a.shape)
print("Positive batch shape:", p.shape)
print("Negative batch shape:", n.shape)


for /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_train_raw_with_paths_clean.csv compute L，saved to: /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_train_raw_with_paths_clean_with_L.csv
for /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_train_raw_with_paths_clean.csv compute L，saved to: /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_train_raw_with_paths_clean_with_L.csv
for /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_test_raw_with_paths_clean.csv compute L，saved to: /content/drive/MyDrive/ColabData/datasets/FEC_project/meta/fec_test_raw_with_paths_clean_with_L.csv
Train dataset len      : 53966
Train eval dataset len : 53966
Test  eval dataset len : 6971
Anchor batch shape  : torch.Size([128, 3, 224, 224])
Positive batch shape: torch.Size([128, 3, 224, 224])
Negative batch shape: torch.Size([128, 3, 224, 224])


In [8]:
# Cell D: Expression embedding & TripletLoss & Optimizer

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

class ExpressionEmbeddingNet(nn.Module):
    def __init__(self, embed_dim=128, pretrained=True):
        super().__init__()
        backbone = models.resnet18(pretrained=pretrained)

        in_dim = backbone.fc.in_features
        backbone.fc = nn.Identity()  # Remove the original classification header

        self.backbone = backbone
        self.proj_head = nn.Sequential(
            nn.Linear(in_dim, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, embed_dim),
        )

    def forward(self, x):
        feat = self.backbone(x)      # [B, in_dim]
        z = self.proj_head(feat)     # [B, embed_dim]
        z = F.normalize(z, p=2, dim=1)  # Unit vectors, convenient for using Euclidean distance/cosine
        return z

embed_dim = 128
model = ExpressionEmbeddingNet(embed_dim=embed_dim, pretrained=True).to(device)

triplet_loss_fn = nn.TripletMarginLoss(margin=0.2, p=2)

optimizer = torch.optim.Adam(
    model.parameters(),
    lr=3e-4,
    weight_decay=1e-4,
)

print(model)


Using device: cuda


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ExpressionEmbeddingNet(
  (backbone): ResNet(
    (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu): ReLU(inplace=True)
    (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (1): BasicBlock(
        (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine

In [9]:
# Evaluate the accuracy of triples

def evaluate_triplet_accuracy(model, loader, device):
    """
    loader: returns (img1, img2, img3, L)
    We process the three pairs of distances d12, d13, d23:
    Find the smallest pair -> map it to a label ∈ {1,2,3} -> compare with the real L
    """
    model.eval()
    correct = 0
    total = 0

    with torch.no_grad():
        for img1, img2, img3, L in loader:
            img1 = img1.to(device)
            img2 = img2.to(device)
            img3 = img3.to(device)
            L    = L.to(device)

            z1 = model(img1)
            z2 = model(img2)
            z3 = model(img3)

            d12 = torch.sum((z1 - z2) ** 2, dim=1)
            d13 = torch.sum((z1 - z3) ** 2, dim=1)
            d23 = torch.sum((z2 - z3) ** 2, dim=1)

            dists = torch.stack([d12, d13, d23], dim=1)   # [B, 3]
            min_idx = torch.argmin(dists, dim=1)          # 0,1,2

            # FEC label mapping:
            # 0 -> pair (1,2) -> L = 3
            # 1 -> pair (1,3) -> L = 2
            # 2 -> pair (2,3) -> L = 1
            idx_to_label = torch.tensor([3, 2, 1], device=device)
            pred_L = idx_to_label[min_idx]

            correct += (pred_L == L).sum().item()
            total   += L.numel()

    acc = correct / total if total > 0 else 0.0
    return acc


In [10]:
#training loop（Triplet Loss baseline)

from time import time
from tqdm.auto import tqdm
import math
import os
import torch

num_epochs = 10
torch.backends.cudnn.benchmark = True

runs_dir = project_dir / "runs" / "stage1_fec_metric"
os.makedirs(runs_dir, exist_ok=True)

# Initialize the best evaluation metrics
best_test_acc = 0.0
best_epoch = 0

for epoch in range(1, num_epochs + 1):
    model.train()
    epoch_loss = 0.0
    n_batches = 0
    t0 = time()

    pbar = tqdm(total=len(train_loader), desc=f"Epoch {epoch}/{num_epochs}")
    for step, (a, p, n) in enumerate(train_loader, start=1):
        a = a.to(device)
        p = p.to(device)
        n = n.to(device)

        za = model(a)
        zp = model(p)
        zn = model(n)

        loss = triplet_loss_fn(za, zp, zn)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        n_batches  += 1

        if step % 100 == 0:
            pbar.set_postfix({"loss": f"{loss.item():.4f}"})

        pbar.update(1)
    pbar.close()

    avg_loss = epoch_loss / max(1, n_batches)
    dt = time() - t0

    # every epoch evaluate triplet accuracy
    train_acc = evaluate_triplet_accuracy(model, train_eval_loader, device)
    test_acc  = evaluate_triplet_accuracy(model, test_eval_loader,  device)

    print(f"\n=== Epoch {epoch}/{num_epochs} finished in {dt/60:.1f} min ===")
    print(f"  Train loss        : {avg_loss:.4f}")
    print(f"  Train triplet acc : {train_acc*100:.2f}%")
    print(f"  Test  triplet acc : {test_acc*100:.2f}%")

    # Check if it is the best model
    if test_acc > best_test_acc:
        best_test_acc = test_acc
        best_epoch = epoch

        # Delete the old best model (if it exists)
        old_best_path = runs_dir / "embedding_encoder_resnet18_triplet_best.pth"
        if os.path.exists(old_best_path):
            os.remove(old_best_path)

        # Save the new best model
        best_model_path = runs_dir / "embedding_encoder_resnet18_triplet_best.pth"
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'test_acc': test_acc,
            'train_acc': train_acc,
            'loss': avg_loss,
        }, best_model_path)
        print(f"  The best model has been saved.! Test acc: {test_acc*100:.2f}%")
        print(f"     Save location: {best_model_path}")
    else:
        print(f"  Current best: epoch {best_epoch}, acc: {best_test_acc*100:.2f}%")

    print()  # Separate each epoch with a blank line

# Training completed
print(f"\n{'='*60}")
print("Training completed!")
print(f"The best model is in the {best_epoch} epoch")
print(f"Best test accuracy: {best_test_acc*100:.2f}%")
print(f"Best model save location: {runs_dir / 'embedding_encoder_resnet18_triplet_best.pth'}")
print(f"{'='*60}")



Epoch 1/10:   0%|          | 0/422 [00:00<?, ?it/s]


=== Epoch 1/10 finished in 4.1 min ===
  Train loss        : 0.1187
  Train triplet acc : 61.97%
  Test  triplet acc : 59.79%
  The best model has been saved.! Test acc: 59.79%
     Save location: /content/drive/MyDrive/CVproject/stage1/runs/stage1_fec_metric/embedding_encoder_resnet18_triplet_best.pth



Epoch 2/10:   0%|          | 0/422 [00:00<?, ?it/s]


=== Epoch 2/10 finished in 4.0 min ===
  Train loss        : 0.0794
  Train triplet acc : 61.77%
  Test  triplet acc : 59.92%
  The best model has been saved.! Test acc: 59.92%
     Save location: /content/drive/MyDrive/CVproject/stage1/runs/stage1_fec_metric/embedding_encoder_resnet18_triplet_best.pth



Epoch 3/10:   0%|          | 0/422 [00:00<?, ?it/s]

...


=== Epoch 8/10 finished in 3.9 min ===
  Train loss        : 0.0319
  Train triplet acc : 60.69%
  Test  triplet acc : 59.46%
  Current best: epoch 6, acc: 61.67%



Epoch 9/10:   0%|          | 0/422 [00:00<?, ?it/s]


=== Epoch 9/10 finished in 3.9 min ===
  Train loss        : 0.0385
  Train triplet acc : 56.14%
  Test  triplet acc : 55.39%
  Current best: epoch 6, acc: 61.67%



Epoch 10/10:   0%|          | 0/422 [00:00<?, ?it/s]


=== Epoch 10/10 finished in 3.9 min ===
  Train loss        : 0.0538
  Train triplet acc : 61.16%
  Test  triplet acc : 58.57%
  Current best: epoch 6, acc: 61.67%


Training completed!
The best model is in the 6 epoch
Best test accuracy: 61.67%
Best model save location: /content/drive/MyDrive/CVproject/stage1/runs/stage1_fec_metric/embedding_encoder_resnet18_triplet_best.pth


In [11]:
#add noise & occlusion to a batch of images

import torch.nn.functional as F
from torchvision import transforms as T

def add_noise_and_occlusion(x, noise_std=0.1, erase_prob=0.5,
                            erase_scale_range=(0.02, 0.2)):
    """
    x: [B, 3, H, W], which is an image after ToTensor + Normalize
    Do two things:
      Gaussian noise: N(0, noise_std^2)
      RandomErasing to simulate a rectangular occlusion block
    """
    # Gaussian noise
    if noise_std > 0:
        noise = noise_std * torch.randn_like(x)
        x_noisy = x + noise
    else:
        x_noisy = x

    # Just crop it a bit to avoid the values being too exaggerated.
    x_noisy = torch.clamp(x_noisy, -3.0, 3.0)

    # Occlusion block (random erasure
    if erase_prob > 0:
        eraser = T.RandomErasing(
            p=erase_prob,
            scale=erase_scale_range,
            ratio=(0.3, 3.3),
            value=0.0,   # Replace the erased area (roughly the darker part) with 0）
            inplace=False,
        )
        for i in range(x_noisy.size(0)):
            x_noisy[i] = eraser(x_noisy[i])

    return x_noisy


In [ ]:
#Train model with Triplet + Consistency regularization

from time import time
from tqdm.auto import tqdm
import os
from pathlib import Path

# First, load a new model from the baseline final checkpoint as initialization.
runs_dir_base = project_dir / "runs" / "stage1_fec_metric"
baseline_ckpt = runs_dir_base / "embedding_encoder_resnet18_triplet_final.pth"

model_cons = ExpressionEmbeddingNet(embed_dim=embed_dim, pretrained=False).to(device)
state_dict = torch.load(baseline_ckpt, map_location=device)
model_cons.load_state_dict(state_dict)
print("Loaded baseline weights from:", baseline_ckpt)

# A consistent version of the optimizer (a slightly smaller lr can be used)
optimizer_cons = torch.optim.Adam(
    model_cons.parameters(),
    lr=1e-4,          # It is slightly smaller than the baseline of 3e-4, serving as fine-tune.
    weight_decay=1e-4,
)

# Training parameters
num_epochs_cons = 10
lambda_cons = 0.2

runs_dir_cons = project_dir / "runs" / "stage1_fec_metric_consistency"
os.makedirs(runs_dir_cons, exist_ok=True)

for epoch in range(1, num_epochs_cons + 1):
    model_cons.train()
    epoch_loss = 0.0
    epoch_triplet = 0.0
    epoch_cons = 0.0
    n_batches = 0

    t0 = time()
    pbar = tqdm(train_loader, desc=f"[Cons] Epoch {epoch}/{num_epochs_cons}", leave=True)

    for anchor, positive, negative in pbar:
        anchor = anchor.to(device, non_blocking=True)
        positive = positive.to(device, non_blocking=True)
        negative = negative.to(device, non_blocking=True)

        # clean version of Triplet Loss
        za = model_cons(anchor)
        zp = model_cons(positive)
        zn = model_cons(negative)

        triplet_loss = triplet_loss_fn(za, zp, zn)

        # Noisy anchor version (only add noise to the anchor, keep p/n clean)
        anchor_noisy = add_noise_and_occlusion(
            anchor,
            noise_std=0.1,
            erase_prob=0.3,
            erase_scale_range=(0.02, 0.2),
        )
        za_noisy = model_cons(anchor_noisy)

        # Consistency loss: 1 - cosine similarity, the closer to 0, the better
        cons_loss = 1.0 - F.cosine_similarity(za, za_noisy, dim=1).mean()

        loss = triplet_loss + lambda_cons * cons_loss

        optimizer_cons.zero_grad(set_to_none=True)
        loss.backward()
        optimizer_cons.step()

        epoch_loss += loss.item()
        epoch_triplet += triplet_loss.item()
        epoch_cons += cons_loss.item()
        n_batches += 1

        pbar.set_postfix({
            "loss": f"{loss.item():.4f}",
            "triplet": f"{triplet_loss.item():.4f}",
            "cons": f"{cons_loss.item():.4f}",
        })

    avg_loss = epoch_loss / max(1, n_batches)
    avg_triplet = epoch_triplet / max(1, n_batches)
    avg_cons = epoch_cons / max(1, n_batches)
    dt = time() - t0

    # Evaluate once on clean using the original evaluate_triplet_accuracy
    train_acc = evaluate_triplet_accuracy(model_cons, train_eval_loader, device)
    test_acc  = evaluate_triplet_accuracy(model_cons, test_eval_loader,  device)

    print(f"\n=== [Cons] Epoch {epoch}/{num_epochs_cons} finished in {dt/60:.1f} min ===")
    print(f"  Total loss        : {avg_loss:.4f}")
    print(f"  Triplet loss      : {avg_triplet:.4f}")
    print(f"  Consistency loss  : {avg_cons:.4f}")
    print(f"  Train triplet acc : {train_acc*100:.2f}%")
    print(f"  Test  triplet acc : {test_acc*100:.2f}%")

    ckpt_path = runs_dir_cons / f"embedding_encoder_resnet18_cons_epoch{epoch}.pth"
    torch.save(model_cons.state_dict(), ckpt_path)
    print("  ✅ Saved consistency-model checkpoint to:", ckpt_path, "\n")

# Save a fixed name for the "eventual consistency model"
final_cons_ckpt = runs_dir_cons / "embedding_encoder_resnet18_cons_final.pth"
torch.save(model_cons.state_dict(), final_cons_ckpt)
print("✅ Saved final consistency encoder to:", final_cons_ckpt)


/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Loaded baseline weights from: /content/drive/MyDrive/CVproject/stage1/runs/stage1_fec_metric/embedding_encoder_resnet18_triplet_final.pth


[Cons] Epoch 1/10:   0%|          | 0/422 [00:00<?, ?it/s]


=== [Cons] Epoch 1/10 finished in 5.3 min ===
  Total loss        : 0.0217
  Triplet loss      : 0.0189
  Consistency loss  : 0.0139
  Train triplet acc : 63.44%
  Test  triplet acc : 59.98%
  ✅ Saved consistency-model checkpoint to: /content/drive/MyDrive/CVproject/stage1/runs/stage1_fec_metric_consistency/embedding_encoder_resnet18_cons_epoch1.pth 



[Cons] Epoch 2/10:   0%|          | 0/422 [00:00<?, ?it/s]

...

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7d9672757b00>
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__
    Exception ignored in: self._shutdown_workers()
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
<function _MultiProcessingDataLoaderIter.__del__ at 0x7d9672757b00>    if w.is_alive():
Traceback (most recent call last):
  File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1654, in __del__

      self._shutdown_workers()  
   File "/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py", line 1637, in _shutdown_workers
      ^^if w.is_alive():
^ ^  ^ ^ ^ ^^ ^^^^^^
^  File "/usr/lib/python3.12/multiprocessing/process.py", line 160, in is_alive
    ^assert self._parent_pid == os.getpid(), 'can only test a child process'
^ ^^^  ^ ^  ^ 
   File "/usr/li


=== [Cons] Epoch 6/10 finished in 5.3 min ===
  Total loss        : 0.0119
  Triplet loss      : 0.0103
  Consistency loss  : 0.0077
  Train triplet acc : 64.18%
  Test  triplet acc : 60.62%
  ✅ Saved consistency-model checkpoint to: /content/drive/MyDrive/CVproject/stage1/runs/stage1_fec_metric_consistency/embedding_encoder_resnet18_cons_epoch6.pth 



[Cons] Epoch 7/10:   0%|          | 0/422 [00:00<?, ?it/s]


=== [Cons] Epoch 7/10 finished in 5.3 min ===
  Total loss        : 0.0108
  Triplet loss      : 0.0096
  Consistency loss  : 0.0063
  Train triplet acc : 59.75%
  Test  triplet acc : 57.08%
  ✅ Saved consistency-model checkpoint to: /content/drive/MyDrive/CVproject/stage1/runs/stage1_fec_metric_consistency/embedding_encoder_resnet18_cons_epoch7.pth 



[Cons] Epoch 8/10:   0%|          | 0/422 [00:00<?, ?it/s]

In [ ]:
#evaluation under synthetic noise/occlusion

@torch.no_grad()
def evaluate_triplet_accuracy_with_noise(model, loader, device, noise_std=0.1, erase_prob=0.3, erase_scale_range=(0.02, 0.2)):
    #Similar to evaluate_triplet_accuracy, except that noise and occlusion are added to all three images.
    model.eval()
    correct = 0
    total = 0

    for img1, img2, img3, L in loader:
        img1 = img1.to(device)
        img2 = img2.to(device)
        img3 = img3.to(device)
        L = L.to(device)

        img1n = add_noise_and_occlusion(img1, noise_std=noise_std,erase_prob=erase_prob,erase_scale_range=erase_scale_range)
        img2n = add_noise_and_occlusion(img2, noise_std=noise_std,erase_prob=erase_prob,erase_scale_range=erase_scale_range)
        img3n = add_noise_and_occlusion(img3, noise_std=noise_std,erase_prob=erase_prob,erase_scale_range=erase_scale_range)

        z1 = model(img1n)
        z2 = model(img2n)
        z3 = model(img3n)

        d12 = ((z1 - z2) ** 2).sum(dim=1)
        d13 = ((z1 - z3) ** 2).sum(dim=1)
        d23 = ((z2 - z3) ** 2).sum(dim=1)

        dists = torch.stack([d12, d13, d23], dim=1)
        min_idx = torch.argmin(dists, dim=1)

        idx_to_label = torch.tensor([3, 2, 1], device=device)
        pred_L = idx_to_label[min_idx]

        correct += (pred_L == L).sum().item()
        total += L.numel()

    acc = correct / total if total > 0 else 0.0
    return acc


In [ ]:
#compare baseline vs consistency model under different noise levels

from pathlib import Path

runs_dir_base = project_dir / "runs" / "stage1_fec_metric"
runs_dir_cons = project_dir / "runs" / "stage1_fec_metric_consistency"

# You can choose for yourself: use the final of the baseline or epoch1/best
baseline_ckpt = runs_dir_base / "embedding_encoder_resnet18_triplet_best.pth"
cons_ckpt     = runs_dir_cons / "embedding_encoder_resnet18_cons_final.pth"
print("Baseline ckpt:", baseline_ckpt)
print("Cons ckpt    :", cons_ckpt)

# Load two model instances
model_base = ExpressionEmbeddingNet(embed_dim=embed_dim, pretrained=False).to(device)
model_base.load_state_dict(torch.load(baseline_ckpt, map_location=device))
model_base.eval()

model_cons_eval = ExpressionEmbeddingNet(embed_dim=embed_dim, pretrained=False).to(device)
model_cons_eval.load_state_dict(torch.load(cons_ckpt, map_location=device))
model_cons_eval.eval()

# Noise intensity setting (you can adjust it again according to the effect)
severity_settings = [
    {"name": "clean",   "noise_std": 0.0,  "erase_prob": 0.0},
    {"name": "level1",  "noise_std": 0.05, "erase_prob": 0.1},
    {"name": "level2",  "noise_std": 0.10, "erase_prob": 0.2},
    {"name": "level3",  "noise_std": 0.15, "erase_prob": 0.3},
    {"name": "level4",  "noise_std": 0.20, "erase_prob": 0.4},
]

print("\n=== Triplet accuracy vs noise level (Test set) ===")
print(f"{'Level':<8} {'Baseline':>10} {'Consistent':>12}")

for cfg in severity_settings:
    name = cfg["name"]
    noise_std = cfg["noise_std"]
    erase_prob = cfg["erase_prob"]

    if noise_std == 0.0 and erase_prob == 0.0:
        acc_base = evaluate_triplet_accuracy(model_base, test_eval_loader, device)
        acc_cons = evaluate_triplet_accuracy(model_cons_eval, test_eval_loader, device)
    else:
        acc_base = evaluate_triplet_accuracy_with_noise(
            model_base, test_eval_loader, device,
            noise_std=noise_std,
            erase_prob=erase_prob,
        )
        acc_cons = evaluate_triplet_accuracy_with_noise(
            model_cons_eval, test_eval_loader, device,
            noise_std=noise_std,
            erase_prob=erase_prob,
        )

    print(f"{name:<8} {acc_base*100:10.2f}% {acc_cons*100:12.2f}%")


Baseline ckpt: /content/drive/MyDrive/CVproject/stage1/runs/stage1_fec_metric/embedding_encoder_resnet18_triplet_final.pth
Cons ckpt    : /content/drive/MyDrive/CVproject/stage1/runs/stage1_fec_metric_consistency/embedding_encoder_resnet18_cons_final.pth

=== Triplet accuracy vs noise level (Test set) ===
Level      Baseline   Consistent
clean         56.89%        60.09%
level1        56.15%        58.82%
level2        53.49%        57.10%
level3        51.93%        55.88%
level4        49.37%        53.46%
